In [1]:
import utils
import pandas as pd
from time import time
from collections import defaultdict
import spacy
import gensim.models.keyedvectors as word2vec
import numpy as np


In [4]:
#initial params
model_path = './data/model_100.bin'
vec_dim = 100
vectors_name_to_save = './data/test_data_vectors'
dataset_file = './dataset/data_preprocessed.csv'

In [5]:
df = pd.read_csv(dataset_file, encoding = 'latin')
df.shape

(2, 1)